## Actions

<i>"By convention, functions that use async transitions are called 'Actions' "</i>

Old way of handling forms was a painfull experience. Please take a look at the below code.

In [ ]:
## old way of handling forms
'use client' 

import React, { ChangeEvent, ChangeEventHandler, FormEvent, useState } from 'react'

export async function updateNameInDB(newName: string) {
  ## Sleep for 1500ms to mimic an API call round trip
  await new Promise(resolve => setTimeout(resolve, 1500))
  if (newName.toLowerCase().includes("error")) {
      throw new Error("Failed to update name")
  }
  localStorage.setItem("name", JSON.stringify(newName))
  return newName
}

export const getNameFromDB = async () => {
  return JSON.parse(localStorage.getItem("name") || "Anonymous user")
}

const Actions = () => {
  const [input, setInput] = useState("")
  const [name, setName] = useState<string>(getNameFromDB())
  
  ## handleChange is required
  function handleChange(event: FormEvent<HTMLInputElement>) {
    const newValue = event.currentTarget.value;
    setInput(newValue)
  }

  async function handleSubmit(event: ChangeEvent<HTMLFormElement>) {
    ## event preventDefault() was necessary
    event.preventDefault()
    try {
      const newName = await updateNameInDB(input)
      setName(newName)
      setInput("")
    } catch (error: any) {
      console.error(error.message)
    }
  }

  return (
    <>
      <p className="username">
        Current user: <span>{name}</span>
      </p>
      <form onSubmit={handleSubmit}> ## onSubmit was necessary
        <input
          type="text"
          value={input} ## value was necessary
          onChange={handleChange} ## onChange was necessary
          required
        />
        <button type="submit">Update</button>
      </form>
    </>
  )
}

export default Actions

#### New way of handling forms with actions

First we remove onSubmit from form and we replace it with action. Next we have to add name parameter to allow React to track the value of this input field

In [ ]:
<form action={handleSubmit}> ## onSubmit was necessary
        <input
          type="text"
          value={input} ## value was necessary
          onChange={handleChange} ## onChange was necessary
          name="username" 
          required
        />
        <button type="submit">Update</button>
      </form>

Action is not an event handler (like it was onSubmit) so we don't get an event as an argument, instead we get formData object (FormData type). Thanks to it we don't have to track it in local React state.

In [ ]:
  async function handleSubmit(formData: FormData) {
    ## removed event.preventDefault()
    try {
      const newName = await updateNameInDB(input)
      setName(newName)
      setInput("")
    } catch (error: any) {
      console.error(error.message)
    }
  }

We can remove const [input, setInput] = useState("") - because it's not necessary to track it.
Now, we have to modify newName variable because it awaits for data from input (which we deleted).

To retrieve the value of specific input field we have to call function formData.get(). Example below

In [ ]:
  async function handleSubmit(formData: FormData) {
    try {
      const newName = await updateNameInDB(formData.get("username"))
      setName(newName)
#       setInput("") we don't need it anymore
    } catch (error: any) {
      console.error(error.message)
    }
  }

We can also remove handleChange function because it's not required anymore, and to distinct action from submitting we can change handleSubmit function to formAction.

Our final code will look like below.

In [ ]:
'use client' 

import React, { ChangeEvent, FormEvent, useState } from 'react'

export async function updateNameInDB(newName: string) {
  // Sleep for 1500ms to mimic an API call round trip
  await new Promise(resolve => setTimeout(resolve, 1500))

  if (newName.toLowerCase().includes("error")) {
      throw new Error("Failed to update name")
  }
  localStorage.setItem("name", JSON.stringify(newName))
  return newName
}

export const getNameFromDB = async () => {
  return JSON.parse(localStorage.getItem("name") || "Anonymous user")
}

const Actions = () => {
  const [name, setName] = useState<string>(getNameFromDB())


  async function formAction(formData: FormData) {
    try {
      const newName = await updateNameInDB(formData.get("username"))
      setName(newName)
    } catch (error: any) {
      console.error(error.message)
    }
  }

  return (
    <>
      <p className="username">
        Current user: <span>{name}</span>
      </p>
      <form action={formAction}>
        <input
          type="text"
          name='username'
          required
        />
        <button type="submit">Update</button>
      </form>
    </>
  )
}

export default Actions